In [ ]:
!pip install groq langchain langchain-core langchain-groq chromadb pypdf gradio sentence-transformers langchain-community langchain-core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 76.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 79.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [ ]:
from langchain_groq import ChatGroq
from langchain_community.document_loaders import TextLoader, PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
# from langchain_community import embeddings
from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.runnables import RunnablePassthrough
# from langchain_core.output_parsers import StrOutputParser
from google.colab import userdata
import os
import time
import textwrap
import gradio as gr

In [ ]:
from langchain.document_loaders import PyPDFLoader

# Specify the correct file path to the PDF
pdf_path = "/content/drive/MyDrive/Resume/College template/717821i211_Dhinesh L.pdf"

# Initialize the PDF loader
loader = PyPDFLoader(pdf_path)

# Load the text from the PDF
text = loader.load()
text


[Document(page_content="DHINESH L\n(+91)9087375428\nDhinesh L\nB.Tech Artificial Intelligence and Data Science \nKarpagam College of Engineering, Coimbatore\nHigher Secondary School - 12th Grade\nSree Ambaal’s Matric Hr Sec School, Perambalur\nSecondary School - 10th Grade \nSree Ambaal’s Matric Hr Sec School, Perambalur\nEXPERIENCEdhineshloganathan77@gmail.com\nDhinesh L\n(2021-2025) \nCGPA 8.02\n(2020-2021) \nPERCENTAGE: 90.8\n(2018-2019) \nPERCENTAGE: 90\nData Scientist Associate\nDatacampLet’sGrowMore\nData Science - Intern\nIntroduction to Machine Learning\nNPTEL\nApache Spark - Programming with Databricks\nData BricksJan 2024 - Feb 2024Jan 2024 - Feb 2024\nJan 2024 - Feb 2024\nNov 2023 - Dec 2023Qlik SenseWe and Data\nPower BI - Intern\nOct 2023 – Nov 2023\n Remote,CoimbatoreApr 2023 – May 2023\n CoimbatoreEDUCATION\nCERTIFICATIONS\nLearn essential statistical concepts and techniques for data analysis, and gain proficiency in working with\nvarious types of raw data. \nBecome skil

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(text)

In [ ]:
from transformers import AutoModel
from langchain.embeddings import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cpu"}

embeddings_hf = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

In [ ]:
vectorstore = Chroma.from_documents(
    documents = chunks,
    collection_name= "groq_embeds",
    embedding = embeddings_hf,
)

retriever = vectorstore.as_retriever()

In [ ]:
from google.colab import userdata
import os
from langchain_groq import ChatGroq

os.environ["GROQ_API_KEY"] = userdata.get('groq_api_key')
llm = ChatGroq(temperature=0, model_name = "mixtral-8x7b-32768" )

In [ ]:
from langchain.chains import RetrievalQA

rag_template = """Answer the question based only on the following context:
{context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(rag_template)
qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=vectorstore.as_retriever(), chain_type_kwargs={"prompt": rag_prompt},
)

In [ ]:
def process_question(user_question):
  response = qa_chain.invoke(user_question)
  full_response = response['result']
  return full_response

In [ ]:
interface = gr.Interface(fn= process_question,
                         inputs= gr.Textbox(lines=2, placeholder="Type your question here"),
                         outputs= gr.Textbox(),
                         title= "Groq Chatbot",
                         description="Ask any question about your documents")

In [ ]:
interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://057a4fff1be8beb4b1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
